https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0#/media/File:Iris_dataset_scatterplot.svg

In [7]:
%pylab inline
import sklearn.datasets
import torch
import torch.nn

random.seed(0)
torch.manual_seed(0)

Populating the interactive namespace from numpy and matplotlib


In [8]:
iris = sklearn.datasets.load_iris()

In [9]:
features = iris.data[:, :2]
labels = iris.target

order = random.permutation(150)

features = features[order, :]
labels = labels[order]

targets = numpy.zeros([150, 3])
targets[arange(150), labels] = 1.0

train_features = features[:100, :]
test_features = features[100:, :]
train_targets = targets[:100]
test_targets = targets[100:]
train_labels = labels[:100]
test_labels = labels[100:]



In [13]:
class IrisNet(torch.nn.Module):
    def __init__(self, n_hidden_neurons):
        
        super(IrisNet, self).__init__()
        self.bn1 = torch.nn.BatchNorm1d(2)
        self.fc1 = torch.nn.Linear(2, n_hidden_neurons)
        self.activ1 = torch.nn.Sigmoid()
        self.bn2 = torch.nn.BatchNorm1d(n_hidden_neurons)
        self.fc2 = torch.nn.Linear(n_hidden_neurons, 3)
        self.sm = torch.nn.Softmax()
        
    def forward(self, x):
        x = self.bn1(x)
        x = self.fc1(x)
        x = self.activ1(x)
        x = self.bn2(x)
        x = self.fc2(x)
#         x = self.activ2(x)
        return x

    def inference(self, x):
        x = self.forward(x)
        x = self.sm(x)
        return x

In [14]:
criterion = torch.nn.CrossEntropyLoss()
    
print train_targets.shape

iris_net = IrisNet(5)

optimizer = torch.optim.Adam(iris_net.parameters(), 
                             lr=1.0e-3)

(100, 3)


In [15]:
batch_size = 10

softmax = torch.nn.Softmax()

for epoch in range(10000):
    order = random.permutation(train_features.shape[0])
    for start_index in range(0, train_features.shape[0], batch_size):
        optimizer.zero_grad()
        
        batch_indice = order[start_index:start_index+batch_size]
        
        batch_features = torch.FloatTensor(train_features[batch_indice, :])
        batch_labels = torch.LongTensor(train_labels[batch_indice])
        
        features_var = torch.autograd.Variable(batch_features)
        targets_var = torch.autograd.Variable(batch_labels)
        
        preds_var = iris_net.forward(features_var)
        
#         print preds_var
#         print targets_var
        
        loss = criterion.forward(preds_var, targets_var)
        loss.backward()
        
        optimizer.step()
        
    test_features = torch.Tensor(test_features)
    features_var = torch.autograd.Variable(test_features)
    test_preds = iris_net.forward(features_var)
    test_preds = softmax(test_preds).data.numpy()
        
    test_preds = test_preds.argmax(axis=1)
    if epoch % 100 == 0:
        print (test_preds == test_labels).sum() / float(test_labels.size)

0.26
0.86
0.82
0.82
0.82
0.82
0.82
0.84
0.82
0.82
0.82
0.8
0.84
0.84
0.84
0.84
0.82
0.82
0.82
0.82
0.82
0.8
0.78
0.78
0.82
0.78
0.78
0.76
0.78
0.78
0.8
0.78
0.78
0.78
0.78
0.78
0.82
0.78
0.82
0.78
0.78
0.78
0.8
0.78
0.78
0.78
0.82
0.8
0.78
0.8
0.82
0.8
0.8
0.8
0.8
0.82
0.82
0.8
0.8
0.8
0.78
0.8
0.8
0.8
0.82
0.8
0.8
0.8
0.8
0.82
0.82
0.82
0.8
0.8
0.8
0.8
0.8
0.8
0.82
0.8
0.8


KeyboardInterrupt: 

# Задание 1
Сделать нейронную сеть с двумя скрытыми слоями (оба сигмоидные). Подобрать количество нейронов в первом и втором слое, которое максимизирует значение accuracy (процент угаданных цветков).

Замените функцию активации с сигмоиды на гиперболический тангенс.
Что изменилось?

# Задание 2
Отцентрируйте и отнормируйте данные, добавьте неиспользованные признаки. Стало ли качество лучше? Что больше повлияло на качество, предобработка данных или увеличение числа рассматриваемых признаков?